In [1]:
from nltk.corpus import stopwords
import nltk
import re
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
import torch
#nltk.download('stopwords')

df = pd.read_csv('Dementia.csv')
df2 = pd.read_csv('Control.csv')

df3 = pd.DataFrame()

# Add the "cookie test" column from df to df3
df3['cookie test'] = df['PAR']

# Add the "dementia" column with value 1 for all rows in df3
df3['dementia'] = 1

# Copy "Error Type 1" and "Error Type 2" columns from df to df3
df3['Error Type 1'] = df['Error Type 1']
df3['Error Type 2'] = df['Error Type 2']
df3['File'] = df['File']
df3['Age'] = df['Age']
df3['Gender'] = df['Gender']
df3['MMSE'] = df['MMSE']

# Create a copy of df2 with the same structure as df3
df2_copy = pd.DataFrame({'cookie test': df2['PAR'], 'dementia': 0})

# Copy "Error Type 1" and "Error Type 2" columns from df2 to df2_copy
df2_copy['Error Type 1'] = df2['Error Type 1']
df2_copy['Error Type 2'] = df2['Error Type 2']
df2_copy['File'] = df2['File']
df2_copy['Age'] = df2['Age']
df2_copy['Gender'] = df2['Gender']
df2_copy['MMSE'] = df2['MMSE']

# Concatenate df3 and df2_copy
df3 = pd.concat([df3, df2_copy], ignore_index=True)

In [2]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   cookie test   550 non-null    object 
 1   dementia      550 non-null    int64  
 2   Error Type 1  550 non-null    int64  
 3   Error Type 2  550 non-null    int64  
 4   File          550 non-null    object 
 5   Age           460 non-null    float64
 6   Gender        548 non-null    object 
 7   MMSE          456 non-null    float64
dtypes: float64(2), int64(3), object(3)
memory usage: 34.5+ KB


In [3]:
df3.head(10)

,cookie test,dementia,Error Type 1,Error Type 2,File,Age,Gender,MMSE
0,mhm . 3609_4282there's a young boy &uh going...,1,7,6,001-2.cha,59.0,male,11.0
1,here's a cookie jar . 0_8778and the lid is o...,1,5,4,003-0.cha,56.0,male,20.0
2,the boy is slipping off the stool . 6536_1030...,1,0,2,005-0.cha,53.0,male,23.0
3,okay he's fallin(g) off a chair [: stool] [* s...,1,1,0,005-2.cha,55.0,male,19.0
4,xxx . [+ exc] 2744_6330can I look at it and ...,1,1,1,007-1.cha,73.0,female,19.0
5,well the girl is telling the boy to get the co...,1,1,0,007-3.cha,75.0,female,15.0
6,oh boy . [+ exc] 2194_3657wowie the boy's go...,1,3,4,010-0.cha,66.0,male,20.0
7,oh boy . [+ exc] 1929_3209alright . [+ exc] ...,1,0,3,010-1.cha,67.0,male,21.0
8,what's happening there ? [+ exc] 1546_2827oh...,1,4,3,010-2.cha,68.0,male,26.0
9,<well the kid> [//] the girl's laughin(g) at h...,1,1,3,010-3.cha,69.0,male,19.0


In [4]:
stop_words = set(stopwords.words('english')) 

text = [] 
cleaned_text = []

for element in df3['cookie test']:
    # Remove the "xxx" at the beginning of the text
    f_text = re.sub(r'^xxx\s*', '', element)
    
    # Remove words within parentheses (including the parentheses)
    f_text = re.sub(r'\([^)]*\)', '', f_text)
    
    # Replace consecutive spaces with a single space, remove non-alphabetic characters, and convert to lowercase
    f_text = re.sub(r"\s+", " ", f_text.strip())
    f_text = re.sub(r"[^a-záéíóúüñ' ]", " ", f_text.lower())
    
    # Remove the word "exc" when it appears as a standalone word
    f_text = re.sub(r"\bexc\b", "", f_text)
    
    # Split the text into words
    docto_text = f_text.split(" ")
    
    # Reconstruct the cleaned text while removing extra spaces
    cleaned_text.append(" ".join(docto_text))
    
    # Create a list of words without stopwords
    x = [word for word in docto_text if word not in stop_words]
    text.append(x)

# Add the 'cleaned_text' column to your DataFrame
#df3['cleaned_text'] = cleaned_text
df3['final_text']= text
vocabulary=[]
voc_siz=[]
lex_voc=[]
for i in range(len(text)):
    voc=list(set(text[i]))
    l_t=len(text[i])
    siz_voc=len(voc)
    vocabulary.append(voc)
    voc_siz.append(siz_voc)
    lex_voc.append(siz_voc/l_t)

In [5]:
print(stop_words)

{'her', 'until', 'isn', 'while', 'mustn', 'weren', "aren't", 'each', 'aren', 'again', "you've", 've', 'wouldn', "doesn't", "you'd", 'are', 'when', "she's", 'than', 'out', 'shouldn', "haven't", 'its', 'he', "that'll", 'herself', 'the', 'through', "shouldn't", 'as', 'won', 'what', 'during', 'very', 'them', 'too', 'for', 'y', 'hasn', 'himself', "mightn't", 'didn', "you're", 'can', 't', 'wasn', 'hadn', 'be', 'haven', 'if', "shan't", 'their', "hadn't", 'ours', 'then', 'myself', 'below', "won't", "didn't", 'above', 'now', 're', "isn't", 'doesn', 'am', 'itself', 'both', 'in', 'there', 'shan', 'few', "couldn't", 'these', 'i', 'were', 'under', 'just', 'only', 'such', 'being', 'was', 'has', 'my', 'at', 'not', 'yourself', 'up', 'mightn', 'by', 'with', 'does', 'is', 'between', 'have', 'theirs', 'whom', 'because', 'yourselves', 'further', 'nor', "don't", 'll', 'here', 'from', 'having', 'down', 'once', 'same', 'd', 'no', 'it', 'before', 'own', 'doing', "it's", 'm', 'they', 'off', 'ma', 'that', 'me',

In [6]:
print("cleaned text",df3['final_text'][0])

cleaned text ['mhm', '', '', '', '', '', '', '', '', '', '', '', '', '', "there's", 'young', 'boy', '', 'uh', 'going', 'cookie', 'jar', '', '', '', '', '', '', '', '', '', '', '', '', '', '', "there's", '', '', '', '', '', 'lit', 'girl', '', '', '', '', '', 'young', 'girl', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', "i'm", 'sayin', "he's", 'boy', 'cause', '', '', '', '', '', '', '', '', 'hard', 'uh', "he's", '', '', '', '', "he's", '', 'c', '', 'cookie', 'jar', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', "there's", '', 'stool', 'already', 'starting', 'toand', 'water', 'sink', '', 'uh', '', 'ev', 'overflowing', 'sink', 'hm', '', '', '', '', 'know', '', '', '', '', '', 'hickey', '', '', '', '', '', 'uh', 'like', '', '', 'uh', 'wife', '', 'g', 'mean', '', '', '', '', '', '', '', 'uh', '', '', '', '', 'motherand', '', '', '', '', '', '', 'uh', '', 'w', '', 'uh', '', 'h', '', '', '', '', '', 'uh', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '

In [7]:
# keys_dem=pd.read_csv(r"C:\Users\marco\OneDrive\Documents\Ijtsi\THESIS\key.csv",delimiter=',')  
# key_words=pd.DataFrame(keys_dem)

# # In[]
# count_key = []

# for cleaned_text in df3['final_text']:
#     h = 0
    
#     for word in cleaned_text.split():
#         m = key_words['Keys'].str.fullmatch(word)
#         h += m.sum()
    
#     count_key.append(h)

keys_dem = pd.read_csv(r"C:\Users\marco\OneDrive\Documents\Ijtsi\THESIS\key.csv", delimiter=',')  
key_words = pd.DataFrame(keys_dem)

count_key = []

for word_list in df3['final_text']:
    h = 0
    
    for word in word_list:
        m = key_words['Keys'].str.fullmatch(word)
        h += m.sum()
    
    count_key.append(h)

In [8]:
model = SentenceTransformer('all-mpnet-base-v2')
corpus_embeddings = model.encode(df3['final_text'])
#pairwise_similarities=cosine_similarity(corpus_embeddings)
#sim=pairwise_similarities[0].tolist()

In [9]:

#all-MiniLM-L6-v2
embedder = SentenceTransformer('all-mpnet-base-v2')
final_storage=[]
for element in df3['final_text']:
    corpus= element
    corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
    corpus_storage=[]
    
    # Query sentences:
    queries = ['children stealing cookies', 'woman doing dishes', 'girl reaching for a cookie','woman not noticing','boy on stool','sink overflowing','stool falling']
    top_k = min(1, len(corpus))
    for query in queries:
        query_embedding = embedder.encode(query, convert_to_tensor=True)
    
        # We use cosine-similarity and torch.topk to find the highest 5 scores
        cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
        top_results = torch.topk(cos_scores, k=top_k)
        for score, idx in zip(top_results[0], top_results[1]):
            results=float(score)
            corpus_storage.append(results)
    final_storage.append(corpus_storage)
# In[]
main_1=[]
main_2=[]
main_3=[]
main_4=[]
main_5=[]
main_6=[]
main_7=[]

for i in range(len(final_storage)):
    m1=final_storage[i][0]
    main_1.append(m1)
    m2=final_storage[i][1]
    main_2.append(m2)
    m3=final_storage[i][2]
    main_3.append(m3)
    m4=final_storage[i][3]
    main_4.append(m4)
    m5=final_storage[i][4]
    main_5.append(m5)
    m6=final_storage[i][5]
    main_6.append(m6)
    m7=final_storage[i][6]
    main_7.append(m7)

In [10]:
count_main=[]
for i in range(len(final_storage)):
    main_used=[]
    for element in final_storage[i]:
        x=0
        if element>=0.50:
            x=1
        main_used.append(x)
    count=sum(main_used)
    count_main.append(count)

In [12]:
data_frames_to_concat = [
    pd.DataFrame({"vocabulary": voc_siz}),
    pd.DataFrame({"lex_div": lex_voc}),
    pd.DataFrame({"key_words": count_key}),
    pd.DataFrame({"main idea 1": main_1}),
    pd.DataFrame({"main idea 2": main_2}),
    pd.DataFrame({"main idea 3": main_3}),
    pd.DataFrame({"main idea 4": main_4}),
    pd.DataFrame({"main idea 5": main_5}),
    pd.DataFrame({"main idea 6": main_6}),
    pd.DataFrame({"main idea 7": main_7}),
    pd.DataFrame({"count main used": count_main}),
   # pd.DataFrame({"sim": sim})
]

# Concatenate DataFrames to df3 column-wise
df3 = pd.concat([df3] + data_frames_to_concat, axis=1)
#df3 = df3.drop(["cookie test", "final_text"], axis=1)

# Save the updated df3 to a CSV file or perform any further operations
df3.to_csv('final_NLP.csv', index=False)

SyntaxError: invalid syntax (906036687.py, line 8)

In [17]:
# Load the first CSV file
file1 = pd.read_csv('final_NLP.csv')

# Load the second CSV file
file2 = pd.read_csv('Pitt-data.csv', header=2)

# Extract the id from the "File" column in file1
file1['id'] = file1['File'].str.extract(r'(\d+)-\d+\.cha').astype(int)

# Merge only the "id," "race," and "educ" columns from file2
merged_df = file1.merge(file2[['id', 'educ']], on='id', how='left')

# Rename the columns as per your requirement
merged_df.rename(columns={'educ': 'Education level'}, inplace=True)

# Save the merged dataframe to a new CSV file
merged_df.to_csv('merged_file.csv', index=False)
